In [2]:
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

import os
from pathlib import Path
import pandas as pd
import nltk
from nltk.corpus import words
import re
from html import unescape

import pycountry
from sklearn.feature_extraction import text 


from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration, OpenAI
import openai

import numpy as np

# Download the words dataset from nltk
nltk.download('words')
nltk.download('punkt')



[nltk_data] Downloading package words to
[nltk_data]     /Users/eryclisrodrigues/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/eryclisrodrigues/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:

df = pd.read_csv(
    r"/Users/eryclisrodrigues/Documents/Eryclis/PhD in Information Science/Courses/IS 597 - HCDS/PROJECT/is-597-hcds/social-media_data/output_sentiment_analysis/final_output_sentiment_RoBERTa_Vader.csv",
   )

In [4]:
df

,pid,type,code,title,score,subreddit,comments,cleaned_comments,vader_sentiment,vader_pos,vader_neu,vader_neg,vader_compound,roBERTa_sentiment_positive,roBERTa_sentiment_negative,roBERTa_sentiment_neutral,preponderant_sentiment
0,p0001,popular,mh_wp,"Apparently, finding interpreters is impossible.",88,deaf,This is absolutely unacceptable. I am an inte...,absolutely unacceptable interpreter worked men...,negative,0.149,0.687,0.165,-0.3989,0.007028,0.921729,0.071243,negative
1,p0002,popular,mh_wp,I hate being deaf,82,deaf,The DMV? I would not have guessed that would b...,dmv would guessed would deaf friendly workplace,positive,0.348,0.652,0.000,0.4939,0.059166,0.601448,0.339386,negative
2,p0003,popular,mh_wp,rant about this community,62,deaf,I'm so sorry you received so much negative ene...,im sorry received much negative energy missed ...,positive,0.209,0.706,0.085,0.9956,0.362453,0.187739,0.449808,neutral
3,p0004,popular,mh_wp,Mental Health Issues In Deaf Community,63,deaf,I don't remember where I saw this so I can't c...,dont remember saw cant cite researching someth...,positive,0.193,0.641,0.166,0.8622,0.069262,0.667851,0.262886,negative
4,p0005,popular,mh_wp,Relationship advice: deaf husband/hearing wife...,57,deaf,"I’m deaf, my husband is hearing. I’m glad you’...",im deaf husband hearing im glad youre proactiv...,positive,0.254,0.598,0.148,0.9301,0.333181,0.277495,0.389324,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,p0289,recent,mh_pa,Lack of seating,28,disability,Not that this is relevant or necessary informa...,relevant necessary information justify complai...,positive,0.139,0.802,0.059,0.4588,0.036293,0.613112,0.350595,negative
248,p0291,recent,mh_pa,‘Disabled’,45,disability,Ayyy I'm also 19 with several disabilities.\n\...,ayyy im also several disability mom genxer com...,negative,0.197,0.594,0.209,-0.3701,0.217780,0.387029,0.395192,neutral
249,p0293,recent,mh_pa,Looking for advice/ opinions on accessible mar...,2,disability,Most adaptive sports organizations began with ...,adaptive sport organization began someone like...,positive,0.315,0.685,0.000,0.9805,0.822026,0.004189,0.173784,positive
250,p0294,recent,mh_pa,Survival Jobs are not disability friendly.,192,disability,I feel this. I can’t stand for basically any p...,feel cant stand basically period time without ...,negative,0.113,0.684,0.202,-0.8651,0.016568,0.869631,0.113801,negative


In [5]:
import re
from html import unescape
from nltk.corpus import words

# Load the list of English words
english_words = set(words.words())

def clean_text(text):
    # 1. Replace \n with space
    #text = text.replace('\n', ' ')
    
    # 2. Remove extra spaces
    #text = re.sub(r'\s+', ' ', text)
    
    # 2. Convert text to lowercase
    text = text.lower()
    
    # 3. Remove leading and trailing spaces
    text = text.strip()
    
    # 4. Remove URLs
    text = re.sub(r'http\S+|www\.\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # 5. Decode HTML entities
    text = unescape(text)
    
    # 6. Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # 7. Remove emails
    text = re.sub(r'\S+@\S+', '', text)
    
    # 8. Remove page references (e.g., Page 1:)
    text = re.sub(r'Page \d+:', '', text, flags=re.IGNORECASE)
    
    # 9. Remove isolated digits (keeping punctuation)
    text = re.sub(r'\b\d+\b', '', text)
    
    # 10. Remove mentions (e.g., @username)
    text = re.sub(r'@\S+', '', text)
    
    # 11. Replace '&' with 'and'
    text = text.replace('&', 'and')
    
    # 12. Remove special characters, keeping basic punctuation
    #text = re.sub(r'[^a-zA-Z0-9.,!?\'\s]', '', text)
    
    # 13. Remove words not in the English vocabulary
    #words_list = text.split()
    #cleaned_words = [word for word in words_list if word.lower() in english_words or re.match(r'[.,!?\'-]', word)]
    #text = ' '.join(cleaned_words)
    
    return text
    


In [15]:
df_copy = df.copy()

In [16]:
df_copy['comments'] = df_copy["comments"].apply(clean_text)

In [17]:
df_copy['comments'][0]

"this is absolutely unacceptable.  i am an interpreter, and i have worked in mental health settings in a supervisory capacity (not as an interpreter).  about % of our residents were deaf.  there was an interpreter at every junction of the residents’ mental health therapy and care, and the faculty and staff (including me, a supervisor) could sign.  a facility like the one i worked at is not the norm, but it certainly exists. where are you in the states? also, look into remote interpreting if there are genuinely no interpreters in your area (unlikely, and not optimal, i know).   yeah, technically a private facility can just say no.  but there are private places that will be accommodating, as well as publicly run options.\ni'm truly sorry about what you're going through. it's bullshit. \n\nmy wife is a deaf therapist, and we live in a deaf mecca (rochester, ny) and she's very frustrated with the mental health system here. there is a serious lack of residential treatment facilities that ca

In [18]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

documents = [Document(page_content=text) for text in df_copy['comments']]

split_texts = []
for doc in documents:
    split_texts.extend(splitter.split_text(doc.page_content))

In [22]:
len(split_texts)

799

### Topic Modeling

In [23]:
# Pre-calculate embeddings
embedding_model = SentenceTransformer("all-mpnet-base-v2")

In [24]:
embeddings = embedding_model.encode(split_texts, show_progress_bar=True)

Batches:   0%|          | 0/25 [00:00<?, ?it/s]

In [25]:
len(embeddings)

799

#### Dimensionality Reduction, Clustering and Vectorizer

In [76]:
umap_model = UMAP(n_neighbors=5, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
vectorizer_model = CountVectorizer(min_df=2, ngram_range=(1, 3), stop_words='english')

In [55]:
# Pre-reduce embeddings for visualization purposes
reduced_embeddings = UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit_transform(embeddings)

In [56]:
len(reduced_embeddings[0])

2

#### Representation models

In [57]:
import os
from dotenv import load_dotenv
from dotenv import load_dotenv
import os

# dotenv_path = os.path.join(os.getcwd(), '.env')
# load_dotenv(dotenv_path)

# api_key = os.getenv('OPENAI_API_KEY')

# Load the .env file
dotenv_path = '/Users/eryclisrodrigues/Documents/Eryclis/PhD in Information Science/Courses/IS 597 - HCDS/PROJECT/is-597-hcds/.env'
load_dotenv(dotenv_path)

# Access the API key
api_key = os.getenv('OPENAI_API_KEY')



In [51]:
openai.api_key = api_key


In [80]:

# KeyBERT
keybert_model = KeyBERTInspired()

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# # GPT-3.5
prompt = """
I have a topic that contains the following documents:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short but highly descriptive topic label of at most 5 words. Make sure it is in the following format:
topic: <topic label>
"""
client = openai.OpenAI()
openai_model = OpenAI(client, model="gpt-3.5-turbo", exponential_backoff=True, chat=True, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    "OpenAI": openai_model,  
    "MMR": mmr_model
}

In [81]:
topic_model = BERTopic(

  # Pipeline models
  embedding_model=embedding_model,  # while we use pre-computed embeddings, internally BERTopic still needs this parameter
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

# Train model
topics, probs = topic_model.fit_transform(split_texts, embeddings)

2024-11-22 20:08:31,687 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-22 20:08:32,817 - BERTopic - Dimensionality - Completed ✓
2024-11-22 20:08:32,818 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-22 20:08:32,830 - BERTopic - Cluster - Completed ✓
2024-11-22 20:08:32,832 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 25/25 [00:16<00:00,  1.51it/s]
2024-11-22 20:09:00,358 - BERTopic - Representation - Completed ✓


In [82]:
# Show topics
freq = topic_model.get_topic_info(); freq

,Topic,Count,Name,Representation,KeyBERT,OpenAI,MMR,Representative_Docs
0,-1,270,-1_health_just_like_mental,"[health, just, like, mental, mental health, pe...","[disability, disabilities, disabled, mental he...",[Challenges of Living with Disabilities],"[health, just, like, mental, mental health, pe...",[yeah your reasons for being on disability is ...
1,0,44,0_deaf_hearing_asl_language,"[deaf, hearing, asl, language, community, heal...","[deaf, hearing world, hearing people, hearing,...",[Deaf Mental Health Services Oppression],"[deaf, hearing, asl, language, community, heal...",[yes deaf community need more mental health th...
2,1,43,1_mental_health_mental health_disability,"[mental, health, mental health, disability, me...","[disability mental health, disability mental, ...",[Mental Health Disability Validity],"[mental, health, mental health, disability, me...",[i personally don't think it's bad. i do under...
3,2,42,2_people_disabled_person_disabled people,"[people, disabled, person, disabled people, ju...","[people disabilities, disabled mental health, ...",[Disability Advocacy and Communication],"[people, disabled, person, disabled people, ju...",[i don’t get offended at all if someone refers...
4,3,29,3_exercise_blind_gym_workout,"[exercise, blind, gym, workout, accessible, bi...","[visually impaired, accessible, sighted, blind...",[Blind Friendly Exercise Activities],"[exercise, blind, gym, workout, accessible, bi...","[in the pool, i often do water jogging with on..."
5,4,28,4_braille_use_computer_screen,"[braille, use, computer, screen, wouldn, acces...","[braille, accessibility, screen readers, scree...",[Braille for Independent Work],"[braille, use, computer, screen, wouldn, acces...",[but you'd think if they weren't going to let ...
6,5,27,5_adhd_disability_impairment_autism,"[adhd, disability, impairment, autism, differe...","[disabilities, disability different, disabilit...",[Neurodiversity and Workplace Inclusion],"[adhd, disability, impairment, autism, differe...","[i consider the terms 'condition', 'impairment..."
7,6,26,6_ssdi_housing_ssi_disability,"[ssdi, housing, ssi, disability, security, dis...","[disability, ssi ssdi, ssi, disabilities, ssdi...",[Social Security Disability Programs],"[ssdi, housing, ssi, disability, security, dis...","[i would call the ssa too, and find out if you..."
8,7,25,7_sighted_blind_blindness_life,"[sighted, blind, blindness, life, little, chil...","[blind people, blindness, blind, sighted, eyes...",[Navigating Life with Visual Impairment],"[sighted, blind, blindness, life, little, chil...",[what made me recover from the darkest times o...
9,8,23,8_wife_pete_years_miss,"[wife, pete, years, miss, ve, think, things, d...","[ex wife, wife, therapy, marriage, advice, mar...",[Coping with Heartbreak and Loss],"[wife, pete, years, miss, ve, think, things, d...","[so... unfortunately, i don't have much advice..."


In [83]:
topic_model.visualize_barchart(top_n_topics=31)

In [84]:
freq['OpenAI']

0           [Challenges of Living with Disabilities]
1           [Deaf Mental Health Services Oppression]
2                [Mental Health Disability Validity]
3            [Disability Advocacy and Communication]
4               [Blind Friendly Exercise Activities]
5                     [Braille for Independent Work]
6           [Neurodiversity and Workplace Inclusion]
7              [Social Security Disability Programs]
8           [Navigating Life with Visual Impairment]
9                  [Coping with Heartbreak and Loss]
10           [Struggles of Living with Disabilities]
11                        [Living with Hearing Loss]
12              [Mental Health Treatment Challenges]
13                   [Hypotonia and Hypermobile EDS]
14        [Emotional Struggle with Vision Diagnosis]
15          [Mobility Challenges in Social Settings]
16                      [Workplace and School Abuse]
17          [Challenges of Living with Disabilities]
18                  [Mental Health Crisis and 

In [85]:
topic_model.topic_labels_

{-1: '-1_health_just_like_mental',
 0: '0_deaf_hearing_asl_language',
 1: '1_mental_health_mental health_disability',
 2: '2_people_disabled_person_disabled people',
 3: '3_exercise_blind_gym_workout',
 4: '4_braille_use_computer_screen',
 5: '5_adhd_disability_impairment_autism',
 6: '6_ssdi_housing_ssi_disability',
 7: '7_sighted_blind_blindness_life',
 8: '8_wife_pete_years_miss',
 9: '9_people_like_just_disability',
 10: '10_hearing_cis_hearing aids_aids',
 11: '11_help_therapist_like_health',
 12: '12_pt_eds_pain_disorder',
 13: '13_vision_eye_migraine_genetic',
 14: '14_park_people_wheelchair_person',
 15: '15_school_teachers_employee_graduate',
 16: '16_people_just_hate_payments',
 17: '17_mental_health_mental health_mentally ill',
 18: '18_public_equipment_transit_transportation',
 19: '19_blind_india_vision_mobility',
 20: '20_treatment_suicide_decide_depression',
 21: '21_blind_using_screen_screen readers',
 22: '22_building_hospital_access_reasonable',
 23: '23_home_shower_w

In [94]:
topic_model.visualize_documents(split_texts, reduced_embeddings=reduced_embeddings, hide_annotations=True, hide_document_hover=False, custom_labels=True)

In [96]:
df_doc_info = topic_model.get_document_info(split_texts)
df_doc_info

,Document,Topic,Name,Representation,KeyBERT,OpenAI,MMR,Representative_Docs,Top_n_words,Probability,Representative_document
0,this is absolutely unacceptable. i am an inte...,0,0_deaf_hearing_asl_language,"[deaf, hearing, asl, language, community, heal...","[deaf, hearing world, hearing people, hearing,...",[Deaf Mental Health Services Oppression],"[deaf, hearing, asl, language, community, heal...",[yes deaf community need more mental health th...,deaf - hearing - asl - language - community - ...,0.908218,False
1,"my wife is a deaf therapist, and we live in a ...",0,0_deaf_hearing_asl_language,"[deaf, hearing, asl, language, community, heal...","[deaf, hearing world, hearing people, hearing,...",[Deaf Mental Health Services Oppression],"[deaf, hearing, asl, language, community, heal...",[yes deaf community need more mental health th...,deaf - hearing - asl - language - community - ...,0.838924,True
2,the dmv? i would not have guessed that would b...,0,0_deaf_hearing_asl_language,"[deaf, hearing, asl, language, community, heal...","[deaf, hearing world, hearing people, hearing,...",[Deaf Mental Health Services Oppression],"[deaf, hearing, asl, language, community, heal...",[yes deaf community need more mental health th...,deaf - hearing - asl - language - community - ...,0.752872,False
3,i'm so sorry you received so much negative ene...,-1,-1_health_just_like_mental,"[health, just, like, mental, mental health, pe...","[disability, disabilities, disabled, mental he...",[Challenges of Living with Disabilities],"[health, just, like, mental, mental health, pe...",[yeah your reasons for being on disability is ...,health - just - like - mental - mental health ...,0.000000,False
4,cart is basically someone typing out every sin...,-1,-1_health_just_like_mental,"[health, just, like, mental, mental health, pe...","[disability, disabilities, disabled, mental he...",[Challenges of Living with Disabilities],"[health, just, like, mental, mental health, pe...",[yeah your reasons for being on disability is ...,health - just - like - mental - mental health ...,0.000000,False
...,...,...,...,...,...,...,...,...,...,...,...
794,feel free to reach out if you ever want a budd...,3,3_exercise_blind_gym_workout,"[exercise, blind, gym, workout, accessible, bi...","[visually impaired, accessible, sighted, blind...",[Blind Friendly Exercise Activities],"[exercise, blind, gym, workout, accessible, bi...","[in the pool, i often do water jogging with on...",exercise - blind - gym - workout - accessible ...,0.500517,False
795,most adaptive sports organizations began with ...,3,3_exercise_blind_gym_workout,"[exercise, blind, gym, workout, accessible, bi...","[visually impaired, accessible, sighted, blind...",[Blind Friendly Exercise Activities],"[exercise, blind, gym, workout, accessible, bi...","[in the pool, i often do water jogging with on...",exercise - blind - gym - workout - accessible ...,1.000000,False
796,i feel this. i can’t stand for basically any p...,16,16_people_just_hate_payments,"[people, just, hate, payments, disability, liv...","[disability effects people, disabilities effec...",[Challenges of Living with Disabilities],"[people, just, hate, payments, disability, liv...",[i'm quite the opposite i can't stay with peop...,people - just - hate - payments - disability -...,1.000000,False
797,"before my disability got really bad, i used to...",9,9_people_like_just_disability,"[people, like, just, disability, understand, p...","[invisible disabilities, disability, people di...",[Struggles of Living with Disabilities],"[people, like, just, disability, understand, p...",[if only that knew they wouldn't be saying tha...,people - like - just - disability - understand...,1.000000,False


In [98]:
df_doc_info.to_csv('/Users/eryclisrodrigues/Documents/Eryclis/PhD in Information Science/Courses/IS 597 - HCDS/PROJECT/is-597-hcds/social-media_data/output_sentiment_analysis/topic_modeling_docs_info.csv', index=False)